# nteract open issue and pr counts
Send a graphql query to GitHub
and work with files for reports.

Supports Python 3.6+

In [1]:
import csv
import json
import os
import pprint

import requests

In [2]:
# get api token and set authorization
api_token = os.environ['GITHUB_API_TOKEN']
headers = {'Authorization': f'token {api_token}'}

In [3]:
# set url to a graphql endpoint
url = 'https://api.github.com/graphql'

In [4]:
# add a json query
query = """
{
  organization(login: "nteract") {
    repositories(first: 80) {
      nodes {
        name
        issues(first: 4) {
          nodes {
            number
            title
            createdAt
          }
        }
      }
    }
  }
}
"""

### Make request and create json and csv files

In [5]:
# submit the request
r = requests.post(url=url, json={'query': query}, headers=headers)

In [6]:
# create a json file from response
with open('data.json', 'w') as f:
    json.dump(r.json(), f)

In [7]:
# unpack the layers of json
nodes = r.json()['data']['organization']['repositories']['nodes']

unpacked = []
for node in nodes:
    unpacked.append(node)

In [8]:
unpacked

[{'issues': {'nodes': [{'createdAt': '2015-05-17T14:09:54Z',
     'number': 1,
     'title': 'Handle input on stdin'},
    {'createdAt': '2015-06-16T14:01:14Z',
     'number': 2,
     'title': 'Installation Problems'},
    {'createdAt': '2015-06-17T15:15:09Z',
     'number': 6,
     'title': 'Cannot build on Windows'},
    {'createdAt': '2015-06-26T03:12:54Z',
     'number': 8,
     'title': 'Display area is launched for kernels that no longer exist'}]},
  'name': 'sidecar'},
 {'issues': {'nodes': [{'createdAt': '2015-05-18T03:34:11Z',
     'number': 1,
     'title': 'Awesome'},
    {'createdAt': '2015-05-18T04:17:49Z',
     'number': 2,
     'title': 'Integrate with autocomplete plus'},
    {'createdAt': '2015-05-18T04:20:26Z',
     'number': 3,
     'title': 'Plots are a bit constrained'},
    {'createdAt': '2015-05-18T13:26:34Z',
     'number': 5,
     'title': "Uncaught TypeError: Cannot read property 'toLowerCase' of undefined"}]},
  'name': 'hydrogen'},
 {'issues': {'nodes': [{'c

In [9]:
headers = ['name', 'issues']

rows = []
for obj in unpacked:
    issue_list = []
    for item in obj['issues']['nodes']:
        issue_list.append({item['number'], item['title'], item['createdAt']})
    new_dict = {'name':obj['name'], 'issues':issue_list}
    rows.append(new_dict)
rows

[{'issues': [{1, '2015-05-17T14:09:54Z', 'Handle input on stdin'},
   {2, '2015-06-16T14:01:14Z', 'Installation Problems'},
   {'2015-06-17T15:15:09Z', 6, 'Cannot build on Windows'},
   {'2015-06-26T03:12:54Z',
    8,
    'Display area is launched for kernels that no longer exist'}],
  'name': 'sidecar'},
 {'issues': [{1, '2015-05-18T03:34:11Z', 'Awesome'},
   {2, '2015-05-18T04:17:49Z', 'Integrate with autocomplete plus'},
   {'2015-05-18T04:20:26Z', 3, 'Plots are a bit constrained'},
   {'2015-05-18T13:26:34Z',
    5,
    "Uncaught TypeError: Cannot read property 'toLowerCase' of undefined"}],
  'name': 'hydrogen'},
 {'issues': [{1, '2015-06-16T01:29:51Z', 'Expose higher level interfaces'},
   {2, '2015-06-18T22:37:14Z', 'support ipython 2/3'},
   {'2015-08-31T11:40:05Z', 4, 'Use the implementation in Hydrogen'},
   {'2016-01-07T07:15:05Z', 5, 'Implement Jupyter Paths directly'}],
  'name': 'jupyter-paths'},
 {'issues': [{1, '2015-12-24T06:00:59Z', 'zmq for everyone'},
   {2, '2015-1

In [10]:
with open('mydata.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(rows)

Check file

In [11]:
%%bash

less mydata.csv

name,issues
sidecar,"[{1, '2015-05-17T14:09:54Z', 'Handle input on stdin'}, {'2015-06-16T14:01:14Z', 2, 'Installation Problems'}, {'Cannot build on Windows', '2015-06-17T15:15:09Z', 6}, {8, 'Display area is launched for kernels that no longer exist', '2015-06-26T03:12:54Z'}]"
hydrogen,"[{1, '2015-05-18T03:34:11Z', 'Awesome'}, {'2015-05-18T04:17:49Z', 2, 'Integrate with autocomplete plus'}, {3, '2015-05-18T04:20:26Z', 'Plots are a bit constrained'}, {""Uncaught TypeError: Cannot read property 'toLowerCase' of undefined"", 5, '2015-05-18T13:26:34Z'}]"
jupyter-paths,"[{1, '2015-06-16T01:29:51Z', 'Expose higher level interfaces'}, {2, '2015-06-18T22:37:14Z', 'support ipython 2/3'}, {'Use the implementation in Hydrogen', 4, '2015-08-31T11:40:05Z'}, {'Implement Jupyter Paths directly', '2016-01-07T07:15:05Z', 5}]"
nteract,"[{'2015-12-24T06:00:59Z', 1, 'zmq for everyone'}, {'2015-12-24T06:06:13Z', 2, 'Scope/Roadmap'}, {'Notebook operations', '2016-01-01T16:14:29Z', 3}, {'Kernelspec operations

## Bring into pandas

In [104]:
import pandas as pd

In [105]:
df = pd.read_csv('mydata.csv')

In [106]:
df.columns

Index(['name', 'issues'], dtype='object')

In [107]:
# df.head()

Generate basic report of total open issues

In [108]:
# df.dtypes

In [109]:
# df.index

In [110]:
# df.values

### Reports

In [111]:
# By repo name
sorted_df = df.sort_values(by=['name'])
sorted_df

,name,issues
5,batchspawner,"[{3, 'Implement testing', '2016-02-24T22:45:18..."
24,binder,"[{'Discuss nomenclature', 3, '2017-08-18T17:57..."
22,binderhub,"[{'Add a README', 1, '2017-05-06T21:19:55Z'}, ..."
1,configurable-http-proxy,"[{'Query by last activity API', 1, '2014-09-17..."
3,dockerspawner,"[{2, '2014-09-27T21:08:00Z', 'Docker layout'},..."
21,helm-chart,"[{1, '2017-04-18T00:04:26Z', 'Allow customizin..."
17,hubshare,"[{'2017-01-27T13:39:47Z', 'implement storage A..."
0,jupyterhub,"[{2, 'use sqlalchemy ORM for state', '2014-08-..."
8,jupyterhub-deploy-docker,[{'Problem with running/authenticating on an E...
11,jupyterhub-deploy-hpc,"[{'Add a jetstream instance for jupyterhub', 2..."


In [138]:
for x in sorted_df:
    name
    issues

    
   

NameError: name 'issues' is not defined

In [113]:
# by open pr count
df.sort_values(by=['prs'], ascending=False)

KeyError: 'prs'

In [ ]:
# output data to a csv
# df.to_csv('issue_report.csv')